Getting data
----

In biomedical contexts, data most often comes from external text files such as spreadsheets. Here we will look at how to import such data into R as a data frame. In order to read spreadsheets correctly, you need to be follow some simple rules when cosntrucitng the table:

### Do this

- A table has column headers and a number of rows and nothing else – it is RECTANGULAR

<img src="good_spreadsheet.png" width="600">

### Not this

- Do not put more than 1 table in a worksheet
- Do not use non-rectangular tables

<img src="bad_spreadsheet.png" width="600">

### Do this

- One cell = one value
- Easy to filter by tube, sample or subject
- Easy to write validation rules or lookup table

<img src="simple_information.png" width="600">

### Not this

- ID column has 3 different values
- Need to do text parsing to recover information – very error prone

<img src="complex_information.png" width="600">

### Round-trip from Excel to CSV and back to Excel

#### Before

- Inofmration in highlighting
- Information in comment notes
- Information in font color
- Merged cells

<img src="before.png" width="600">

### After

- Comments are lost
- Highlighting is lost
- Bad cell formatting is lost
- Merged cells become missing information

<img src="after.png" width="600">


Other suggestions
----

- Use a lookup table rather than typing if possible to avoid errors due to typos
- Use a special marker to indicate misssing values - do not use 0 or 999 etc
- Do not keeep multiple copies of the same spredsheet 
- If you must keep multiple copies, make sure you version them clearly in the fileanme
- Excel is OK if you use almost NONE of its features!

In [58]:
# Don't show warnign messages to keep interface clean
options(warn = -1)

Reading data from a spreadsheet
----

In [59]:
library(RCurl)
library(gdata)

In [60]:
songs.url <- "http://www.acclaimedmusic.net/Current/top_6000_songs_140727.xls"

In [135]:
songs <- read.xls(songs.url, stringsAsFactors=FALSE)

In [136]:
dim(songs)

[1] 7662   18

### Cleaning the song and album dataframes

In [137]:
head(songs, 3)

,ID,PLACE.2014.JUL.27,Artist,Song,Year,X,PLACE.2008.NOV.29,PLACE.2011.NOV.10,PLACE.2013.JUN.21,PLACE.2014.JUL.27.1,X.1,CHANGE.2008.2011,CHANGE.2011.2013,CHANGE.2013.2014,X.2,Trend,X.3,X.4
1,991,1,Bob Dylan,Like a Rolling Stone,1965,,1,1,1,1,,0,0,0,,0,0,0
2,182,2,Nirvana,Smells Like Teen Spirit,1991,,4,3,3,2,,1,0,1,,0.25,0,0.333333333333333
3,961,3,The Beach Boys,Good Vibrations,1966,,3,4,4,3,,-1,0,1,,-0.25,0,0.25


In [138]:
colnames(songs)

[1] "ID"                  "PLACE.2014.JUL.27"   "Artist"             
 [4] "Song"                "Year"                "X"                  
 [7] "PLACE.2008.NOV.29"   "PLACE.2011.NOV.10"   "PLACE.2013.JUN.21"  
[10] "PLACE.2014.JUL.27.1" "X.1"                 "CHANGE.2008.2011"   
[13] "CHANGE.2011.2013"    "CHANGE.2013.2014"    "X.2"                
[16] "Trend"               "X.3"                 "X.4"

#### Let's just keep the first 5 columns

In [139]:
keep <- c('ID', 'PLACE.2014.JUL.27', 'Artist', 'Song', 'Year')
songs <- songs[,keep]

#### Shorten the 2nd column name to 'Place'

In [140]:
colnames(songs)[2] <- 'Place'

In [141]:
head(songs, 3)

,ID,Place,Artist,Song,Year
1,991,1,Bob Dylan,Like a Rolling Stone,1965
2,182,2,Nirvana,Smells Like Teen Spirit,1991
3,961,3,The Beach Boys,Good Vibrations,1966


#### There are only 6000 ranked songs - get rid of the rest

In [142]:
songs[5998:6003,]

,ID,Place,Artist,Song,Year
5998,6538,5998,George Michael,Too Funky,1992
5999,6196,5999,Eddie Cochran,Three Steps to Heaven,1960
6000,6672,6000,Daryl Hall & John Oates,Kiss on My List,1981
6001,5057,,!!!,Hello? Is This Thing On?,2004
6002,5058,,!!!,Pardon My Freedom,2004
6003,8720,,\Tennessee\ Ernie Ford,Shot Gun Boogie,1950


In [143]:
songs <- songs[1:6000,]

In [144]:
tail(songs, 3)

,ID,Place,Artist,Song,Year
5998,6538,5998,George Michael,Too Funky,1992
5999,6196,5999,Eddie Cochran,Three Steps to Heaven,1960
6000,6672,6000,Daryl Hall & John Oates,Kiss on My List,1981


#### Take a closer look at the type of each column

In [145]:
sapply(songs, class)

ID       Place      Artist        Song        Year 
"character" "character" "character" "character" "character"

#### Change types to more appropriate classes so that we can manipulate them

In [105]:
songs$Place <- as.numeric(songs$Place)

In [148]:
songs[, c('ID', 'Place', 'Year')] <- sapply(songs[, c('ID', 'Place', 'Year')], as.numeric)

In [149]:
sapply(songs, class)

ID       Place      Artist        Song        Year 
  "numeric"   "numeric" "character" "character"   "numeric"

Work!
----

There is a spreadsheet of the top 5,000 albums available at http://www.acclaimedmusic.net/Current/top_3000_albums_140727.xls

Create a dataframe called albums that holds the contents of this spreadsheet.

How many rows and columns are there in the datafraem?

Trim the albums dataframe so that it only contains the top 5,000 albusm and the first 5 columns.

Convert the types of each column similar to what was done for the songs dataframe.

Getting information from dataframes
----

### Indexing

In [150]:
# Gettting one row
songs[100,]

,ID,Place,Artist,Song,Year
100,1055,100,The Beatles,She Loves You,1963


In [151]:
# getting rows from one column
songs[10:13, 3]

[1] "Chuck Berry"       "Marvin Gaye"       "Otis Redding"     
[4] "Bruce Springsteen"

In [152]:
# using column names
songs[10:13, 'Artist']

[1] "Chuck Berry"       "Marvin Gaye"       "Otis Redding"     
[4] "Bruce Springsteen"

#### Using conditions to filter rows

In [153]:
songs[(songs$Place >= 10) & (songs$Place < 13),]

,ID,Place,Artist,Song,Year
10,1201,10,Chuck Berry,Johnny B. Goode,1958
11,811,11,Marvin Gaye,What's Going On,1971
12,902,12,Otis Redding,(Sittin' On) The Dock of the Bay,1968


In [154]:
# What is James Taylor's highest ranked song?
songs[match("James Taylor", songs$Artist),]

,ID,Place,Artist,Song,Year
401,860,401,James Taylor,Fire and Rain,1970


In [155]:
# Find all songs fro groups or artists with pink in their name
idx <- grep("pink", songs$Artist, ignore.case=TRUE)
songs[idx,]

,ID,Place,Artist,Song,Year
197,948,197,Pink Floyd,See Emily Play,1967
319,4888,319,Ariel Pink,Round and Round,2010
398,2589,398,Pink Floyd,Wish You Were Here,1975
693,1839,693,Pink Floyd,Comfortably Numb,1979
712,586,712,Pink Floyd,"Another Brick in the Wall, Part 2",1979
919,762,919,Pink Floyd,Money,1973
1108,4825,1108,The Big Pink,Dominos,2009
1311,1691,1311,Pink Floyd,Arnold Layne,1967
1388,3702,1388,Pink Floyd,Astronomy Domine,1967
1771,1771,1771,Pink Floyd,Shine On You Crazy Diamond,1975


In [156]:
# How many songs does Pink Floyd have in the songs dataframe?
sum(songs$Artist == "Pink Floyd")

[1] 13

Work!
----

List the songs in positions 1000, 2000, 3000, 4000, 5000 and 6000.

How mnay songs have "blue" in the song title?

How many top songs were there in total in the years 2012, 2013 and 2014?

Getting summary statistics
----

In [110]:
summary(songs)

       ID           Place         Artist              Song          
 Min.   :   4   Min.   :   5   Length:6000        Length:6000       
 1st Qu.:1550   1st Qu.:1505   Class :character   Class :character  
 Median :3242   Median :3004   Mode  :character   Mode  :character  
 Mean   :3507   Mean   :3004                                        
 3rd Qu.:5458   3rd Qu.:4504                                        
 Max.   :7662   Max.   :6004                                        
      Year           
 Min.   :1897-06-24  
 1st Qu.:1969-06-24  
 Median :1983-06-24  
 Mean   :1982-11-09  
 3rd Qu.:1998-06-24  
 Max.   :2013-06-24  

In [ ]:
byYear <- xtabs(~ Year, songs)

In [181]:
tail(byYear, 3)

Year
2011 2012 2013 
 102  101   95

In [254]:
tail(rowSums(byArtistYear), 3)

Zomby Zoot Woman     ZZ Top 
         1          2          5

In [252]:
byArtist <- xtabs(~ Artist, songs)
popular.artist <- byArtist[order(-byArtist)]
popular.artist[1:10]

Artist
       The Beatles The Rolling Stones          Bob Dylan        David Bowie 
                78                 53                 48                 33 
 Bruce Springsteen      Elvis Presley                 U2            Madonna 
                28                 28                 26                 25 
            Prince             R.E.M. 
                25                 24

In [246]:
head(songs)

,ID,Place,Artist,Song,Year
1,991,1,Bob Dylan,Like a Rolling Stone,1965
2,182,2,Nirvana,Smells Like Teen Spirit,1991
3,961,3,The Beach Boys,Good Vibrations,1966
4,992,4,The Rolling Stones,(I Can't Get No) Satisfaction,1965
5,2517,5,The Beatles,A Day in the Life,1967
6,901,6,Marvin Gaye,I Heard It Through the Grapevine,1968


In [183]:
byArtistYear <- xtabs(~ Artist + Year, songs)

In [259]:
ncols <- ncol(byArtistYear)
tail(byArtistYear[, (ncols-10):ncols], 3)

,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
Zomby,0,0,0,0,0,0,0,0,1,0,0
Zoot Woman,1,0,0,0,0,0,0,0,0,0,0
ZZ Top,0,0,0,0,0,0,0,0,0,0,0


In [260]:
tail(colSums(byArtistYear), 3)

2011 2012 2013 
 102  101   95

In [261]:
tail(apply(byArtistYear, 2, sum), 3)

2011 2012 2013 
 102  101   95

Work!
----

Combining dataframes
----

In [262]:
albums.url <- "http://www.acclaimedmusic.net/Current/top_3000_albums_140727.xls"
albums <- read.xls(albums.url, , stringsAsFactors=FALSE)

In [263]:
head(albums, 3)

,i,PLACE.2014.JUL.27,Artist,Album,Year,X,PLACE.2008.NOV.30,PLACE.2009.NOV.08,PLACE.2010.NOV.28,PLACE.2013.JUN.21,PLACE.2014.JUL.27.1,X.1,CHANGE.2008.2009,CHANGE.2009.2010,CHANGE.2010.2013,CHANGE.2013.2014,X.2,Trend,X.3,X.4,X.5
1,63,1,The Beatles,Revolver,1966,NA,2,2,2,1,1,NA,0,0,1,0,NA,0,0,0.5,0
2,62,2,The Beach Boys,Pet Sounds,1966,NA,1,1,1,2,2,NA,0,0,-1,0,NA,0,0,-0.5,0
3,811,3,Nirvana,Nevermind,1991,NA,3,3,3,3,3,NA,0,0,0,0,NA,0,0,0,0


In [264]:
albums <- albums[1:5000, 1:5]

In [272]:
(col.names <- colnames(songs))
col.names[4] <- 'Album'
(colnames(albums) <- col.names)

[1] "ID"     "Place"  "Artist" "Song"   "Year"

[1] "ID"     "Place"  "Artist" "Album"  "Year"

In [273]:
head(albums, 3)

,ID,Place,Artist,Album,Year
1,63,1,The Beatles,Revolver,1966
2,62,2,The Beach Boys,Pet Sounds,1966
3,811,3,Nirvana,Nevermind,1991


In [276]:
merge(x=songs[1:10,], y=albums[1:10,], by=c('Artist', 'Year'))

,Artist,Year,ID.x,Place.x,Song,ID.y,Place.y,Album
1,Nirvana,1991,182,2,Smells Like Teen Spirit,811,3,Nevermind
2,The Beach Boys,1966,961,3,Good Vibrations,62,2,Pet Sounds
3,The Beatles,1967,931,8,Strawberry Fields Forever,92,5,Sgt. Pepper's Lonely Hearts Club Band
4,The Beatles,1967,2517,5,A Day in the Life,92,5,Sgt. Pepper's Lonely Hearts Club Band


Work!
----

Work!
----